In [14]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [15]:
data=pd.read_csv('../merged/len3_ndb_agg_blp_DropNever.csv',encoding="shift-jis",index_col=0)
data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_399.0,id_441.0,id_449.0,id_625.0,id_629.0,id_799.0,q_share,r_share,generic_share_q,generic_share_r
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014,1,...,0,0,0,0,0,0,0.007258,0.002068,0.236142,0.053889
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014,1,...,0,0,0,0,0,0,0.002987,0.001507,0.236142,0.053889
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014,0,...,0,0,0,0,0,0,0.014653,0.004175,0.236142,0.053889
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014,0,...,0,0,0,0,0,0,0.007997,0.004036,0.236142,0.053889
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014,1,...,0,0,0,0,0,0,0.001557,0.002136,0.236142,0.053889


In [16]:
data.columns[77:102]

Index(['id_114.0', 'id_121.0', 'id_123.0', 'id_131.0', 'id_132.0', 'id_218.0',
       'id_223.0', 'id_225.0', 'id_229.0', 'id_231.0', 'id_232.0', 'id_239.0',
       'id_245.0', 'id_252.0', 'id_259.0', 'id_264.0', 'id_265.0', 'id_332.0',
       'id_339.0', 'id_399.0', 'id_441.0', 'id_449.0', 'id_625.0', 'id_629.0',
       'id_799.0'],
      dtype='object')

In [17]:
for target in ["lag_mean_price","lag_sum_quantity"]:
    data_est=data.dropna(subset=[target])
    ids=data_est.iloc[:,77:102]
    X=data_est[["year"]+list(ids.columns)]
    y=data_est[target]
    # X = sm.add_constant(X)  # 定数項を追加
    model = sm.OLS(y, X).fit()

    # モデルの要約を表示
    print(model.summary())
    missing=data[data[target].isnull()]
    missing=missing[["year"]+list(ids.columns)]
    # missing["const"]=1
    predicted_values=model.predict(missing)
    # predicted_values
    missing[target]=predicted_values
    data.update(missing)

                            OLS Regression Results                            
Dep. Variable:         lag_mean_price   R-squared:                       0.920
Model:                            OLS   Adj. R-squared:                  0.920
Method:                 Least Squares   F-statistic:                 1.129e+04
Date:                Sat, 09 Sep 2023   Prob (F-statistic):               0.00
Time:                        21:56:03   Log-Likelihood:            -1.5474e+05
No. Observations:               24448   AIC:                         3.095e+05
Df Residuals:                   24422   BIC:                         3.097e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
year          -4.1325      0.433     -9.534      0.0

In [18]:
# data.columns
data.to_csv('../merged/len3_ndb_agg_blp_imputed_lags.csv',encoding="shift-jis")

In [19]:
data.shape

(25798, 106)